In [1]:
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from scipy.special import expit
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed, parallel_backend
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import loadmat
os.chdir("data")
data = loadmat("data.mat")
os.chdir("..")
seed = 42

In [2]:
class LogisticRegression:
    def __init__(self, learning_rate =0.01, iterations = 1000, regularization=0.1, stochastic=True, divide_lr = False):
        """"
        L2 regularized logistic regression model
        """
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.regularization = regularization
        self.stochastic = stochastic
        self.divide_lr = divide_lr
        return
    def normalize(self, X):
        """
        Normalizes ny mean and standard deviation
        """
        mu = np.mean(X, axis=0)
        std = np.std(X,axis=0)
        X_norm = (X-mu)/std
        return X_norm
    def add_bias(self, X):
        n = X.shape[0]
        X = np.hstack((np.ones((n, 1)), X))  # Add bias term
        return X
    def fit(self, X, Y, w=None):
        """"
        Use w to set an inital weight vector if desired
        """
        X_norm = self.normalize(X)
        X_norm = self.add_bias(X_norm)
        m, n = X_norm.shape
        if w is not None:
            self.w = np.full((n,1), w)
        else:
            self.w = np.zeros((n, 1))  # Initialize weights

        if not self.stochastic:
            self.batch_gradient_descent(X_norm,Y)
        else:
            self.stochastic_gradient_descent(X_norm,Y)
        return
    def compute_loss(self, X, Y, external=False):
        """
        Compute mean cross-entropy loss
        """
        if external:
            X = self.normalize(X)
            X = self.add_bias(X)
        y_hat = expit(X @ self.w)
        m = len(Y)
        loss = (-1/m) * np.sum(Y * np.log(y_hat) + (1 - Y) * np.log(1 - y_hat))
        return loss
    def predict_proba(self, X):
        """"
        returns probabilities under logistic function
        """
        return expit(X @ self.w)
    def batch_gradient_descent(self, X, Y):
        """"
        Batch Gradient descent
        """
        for _ in tqdm(range(self.iterations), desc=f"Batch GD with {self.iterations} iterations: "):
            y_hat = self.predict_proba(X)
            n = X.shape[0]
            gradient = (1/n) * X.T @ (y_hat - Y)
            gradient += (self.regularization/n) * np.vstack(([0], self.w[1:])) 
            self.w -= self.learning_rate * gradient
    def predict(self,X,threshold=0.5):
        X_norm = self.normalize(X)
        X_norm = self.add_bias(X_norm)
        preds = (self.predict_proba(X_norm) >= threshold).astype(int)
        return np.ravel(preds)
    def stochastic_gradient_descent(self,X,Y):
        lr = self.learning_rate
        loss_history = []
        s = seed
        for i in tqdm(range(1, self.iterations +1), desc=f"SGD with {self.iterations} iterations: "):
            s+=1
            rng = np.random.RandomState(s)
            inds = rng.permutation(X.shape[0])
            X_shuffle = X[inds]
            Y_shuffle = Y[inds]
            for j in range(1):  #allows for an easy switch to minibatch GD
                x_i = X_shuffle[j].reshape(1, -1)  #
                y_i = Y_shuffle[j]

                y_hat = expit(x_i @ self.w)  
                error = y_hat - y_i
                self.w[1:] = (1 - lr * self.regularization) * self.w[1:] - lr * error * x_i.T[1:]
                self.w[0] -= lr * error.item() 
                if self.divide_lr:
                    lr = self.learning_rate / i
        return
"""             loss_history.append(self.compute_loss(X,Y))
            if epoch > 10 and abs(loss_history[-1] - loss_history[-2]) < 1e-7:
                print(f"Early stopping at epoch {epoch}")
                break
        return """


'             loss_history.append(self.compute_loss(X,Y))\n            if epoch > 10 and abs(loss_history[-1] - loss_history[-2]) < 1e-7:\n                print(f"Early stopping at epoch {epoch}")\n                break\n        return '

In [5]:
X = data.get('X')
Y = data.get('y')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)
model = LogisticRegression(stochastic=False)
model.fit(X_train, y_train)
y_preds = model.predict(X_test)
print(classification_report(y_test, y_preds))

Batch GD with 1000 iterations: 100%|██████████| 1000/1000 [00:00<00:00, 2759.74it/s]

              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99       787
         1.0       0.93      0.99      0.95       213

    accuracy                           0.98      1000
   macro avg       0.96      0.98      0.97      1000
weighted avg       0.98      0.98      0.98      1000



In [6]:

model = LogisticRegression(stochastic=True)
model.fit(X_train, y_train)
y_preds = model.predict(X_test)
print(classification_report(y_test, y_preds))

SGD with 1000 iterations: 100%|██████████| 1000/1000 [00:00<00:00, 1312.61it/s]

              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99       787
         1.0       0.96      0.99      0.97       213

    accuracy                           0.99      1000
   macro avg       0.98      0.99      0.98      1000
weighted avg       0.99      0.99      0.99      1000

